# Index Prefix on Compound Indexes

---
### Connect to local server

---

In [1]:
# Importing the required libraries
import pymongo

import pprint as pp
pp.sorted = lambda x, key=None: x

In [2]:
# Connect to local host
client = pymongo.MongoClient("mongodb://localhost:27017/")

In [3]:
# Connect to database
db = client['nyc']

In [4]:
# Sample document
pp.pprint(
    db.airbnb.find_one()
)

{'_id': ObjectId('60c21bf5b653d40e79b4a7d0'),
 'accom_id': 2595,
 'description': 'Skylit Midtown Castle',
 'host': {'id': 2845,
          'name': 'Jennifer',
          'listings_count': 3,
          'neighbourhood_list': ['Midtown', "Hell's Kitchen"]},
 'neighbourhood': {'name': 'Midtown', 'group': 'Manhattan'},
 'location': {'type': 'Point', 'coordinates': [-73.98559, 40.75356]},
 'room_type': 'Entire home/apt',
 'price': 150,
 'minimum_nights': 30,
 'reviews': {'number_of_reviews': 48,
             'last_review': datetime.datetime(2019, 11, 4, 0, 0),
             'reviews_per_month': 0.35},
 'availability_365': 365}


----
**Drop previous indexes.**

---

In [5]:
# Drop indexes
db.airbnb.drop_indexes()

---

### [Index Prefix](https://docs.mongodb.com/manual/core/index-compound/#prefixes)

- Continuous subset index of Compound index.

- Index fields are passed in order from left to right.

**Compound index :-** `{ "price": 1, "reviews.number_of_reviews": -1, availability_365: -1}`

![image.png](Images/prefix_index.png)

**Index prefix :-**

- `{ "price": 1}`

![](Images/index_prefix_2.png)

- `{ "price": 1, "reviews.number_of_reviews": -1}`

![](Images/index_prefix_3.png)

**Not Index prefix :-**

- `{ "price": 1, "availability_365": -1}`

- `{ "availability_365": 1}`

- `{ "reviews.number_of_reviews": -1, "availability_365": -1}`


**MongoDB will support queries on :-**

- `price`

- `price` and `reviews.number_of_reviews`

- `price` and `reviews.number_of_reviews` and `availability_365`



---

In [6]:
# Create compound index

db.airbnb.create_index(
                        # Compound index
                        [
                            ('price', pymongo.ASCENDING),
                            ('reviews.number_of_reviews', pymongo.DESCENDING),
                            ('availability_365', pymongo.DESCENDING)
                        ],
                        # Index name
                        name = 'price_reviews_availability')

'price_reviews_availability'

----
Look for documents where `150 < price <170`, `reviews.number_of_reviews > 50` and `availability_365 > 31`. 

---

In [7]:
# Query

pp.pprint(
            db.airbnb.find({
                                'price': {
                                            '$gt': 150,
                                            '$lt': 170
                                        },
                                'reviews.number_of_reviews': {'$gt': 50},
                                'availability_365': {'$gt': 31}
                          })\
                     .explain()['executionStats']
    )

{'executionSuccess': True,
 'nReturned': 152,
 'executionTimeMillis': 5,
 'totalKeysExamined': 220,
 'totalDocsExamined': 152,
 'executionStages': {'stage': 'FETCH',
                     'nReturned': 152,
                     'executionTimeMillisEstimate': 0,
                     'works': 220,
                     'advanced': 152,
                     'needTime': 67,
                     'needYield': 0,
                     'saveState': 0,
                     'restoreState': 0,
                     'isEOF': 1,
                     'docsExamined': 152,
                     'alreadyHasObj': 0,
                     'inputStage': {'stage': 'IXSCAN',
                                    'nReturned': 152,
                                    'executionTimeMillisEstimate': 0,
                                    'works': 220,
                                    'advanced': 152,
                                    'needTime': 67,
                                    'needYield': 0,
              

---
Using the Compound Index `price_reviews` we will be able to query on the `price` field alone as well.

---

In [8]:
# Query

pp.pprint(
            db.airbnb.find({
                                'price': {
                                            '$gt': 150,
                                            '$lt': 170
                                        }
                          })\
                     .explain()['executionStats']
    )

{'executionSuccess': True,
 'nReturned': 1215,
 'executionTimeMillis': 2,
 'totalKeysExamined': 1215,
 'totalDocsExamined': 1215,
 'executionStages': {'stage': 'FETCH',
                     'nReturned': 1215,
                     'executionTimeMillisEstimate': 0,
                     'works': 1216,
                     'advanced': 1215,
                     'needTime': 0,
                     'needYield': 0,
                     'saveState': 1,
                     'restoreState': 1,
                     'isEOF': 1,
                     'docsExamined': 1215,
                     'alreadyHasObj': 0,
                     'inputStage': {'stage': 'IXSCAN',
                                    'nReturned': 1215,
                                    'executionTimeMillisEstimate': 0,
                                    'works': 1216,
                                    'advanced': 1215,
                                    'needTime': 0,
                                    'needYield': 0,
      

----
Look for documents where `150 < price <170` and `reviews.number_of_reviews > 50`.

---

In [9]:
# Query

pp.pprint(
            db.airbnb.find({
                                'price': {
                                            '$gt': 150,
                                            '$lt': 170
                                        },
                                'reviews.number_of_reviews': {'$gt': 50}
                            })\
                     .explain()['executionStats']
    )

{'executionSuccess': True,
 'nReturned': 201,
 'executionTimeMillis': 0,
 'totalKeysExamined': 221,
 'totalDocsExamined': 201,
 'executionStages': {'stage': 'FETCH',
                     'nReturned': 201,
                     'executionTimeMillisEstimate': 0,
                     'works': 221,
                     'advanced': 201,
                     'needTime': 19,
                     'needYield': 0,
                     'saveState': 0,
                     'restoreState': 0,
                     'isEOF': 1,
                     'docsExamined': 201,
                     'alreadyHasObj': 0,
                     'inputStage': {'stage': 'IXSCAN',
                                    'nReturned': 201,
                                    'executionTimeMillisEstimate': 0,
                                    'works': 221,
                                    'advanced': 201,
                                    'needTime': 19,
                                    'needYield': 0,
              

----
So we do not need to create an unnecessary index for `price` field alone or for `price` and `reviews.number_of_review`. We can create a Compound Index that can work for both types of queries.

However, **Index Prefixes work in order from left to right in a continuous manner**. So, if we try to query on the `reviews.number_of_reviews`, we will not be able to utilize the index.

----

In [10]:
# Query

pp.pprint(
            db.airbnb.find({
                                'reviews.number_of_reviews': {'$gt': 50}
                            }).explain()['executionStats']
    )

{'executionSuccess': True,
 'nReturned': 4971,
 'executionTimeMillis': 28,
 'totalKeysExamined': 0,
 'totalDocsExamined': 36905,
 'executionStages': {'stage': 'COLLSCAN',
                     'filter': {'reviews.number_of_reviews': {'$gt': 50}},
                     'nReturned': 4971,
                     'executionTimeMillisEstimate': 1,
                     'works': 36907,
                     'advanced': 4971,
                     'needTime': 31935,
                     'needYield': 0,
                     'saveState': 36,
                     'restoreState': 36,
                     'isEOF': 1,
                     'direction': 'forward',
                     'docsExamined': 36905},
 'allPlansExecution': []}


---
Index prefixes work from left to right in a continuous manner. Since `reviews.number_of_reviews` breaks that chain, MongoDB had to use the COLLSCAN or default `_id` field as index.

**Advantage :-**

*`If you sometimes query on only one key and at other times query on that key combined with a second key, then creating a compound index is more efficient than creating a single-key index.`*

---

----
### Exercise -

Consider the following queries.

- How many accomodations had `room_type` as `Private room`, `price < 100`, and `reviews.reviews_per_month > 0.5' ?

- How many accomodations had `room_type` as `Private room` and `price < 100`?

- How many accomodations had `room_type` as `Private room` or `Shared room`?

What index did you use? Were compound indexes of any use?

-----